<a href="https://colab.research.google.com/github/sankethgadadinni/RoadClassification/blob/main/RoadClassifty.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Sat Jan 23 09:13:15 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   58C    P8    11W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install kaggle


In [3]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"sankethgadadinni","key":"24af2328238021562621f8ae9f36d39f"}'}

In [4]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [5]:
!kaggle datasets download -d virenbr11/pothole-and-plain-rode-images

 97% 233M/241M [00:02<00:00, 141MB/s]
100% 241M/241M [00:02<00:00, 111MB/s]


In [6]:
from zipfile import ZipFile 
filename = 'pothole-and-plain-rode-images.zip'

with ZipFile(filename,'r') as zip:
  zip.extractall()
  print('Done')

Done


In [7]:
import os
import numpy as np
import cv2
import time
import copy
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import datasets,transforms
from torchvision import models
from torch.utils.data import DataLoader,Dataset

In [9]:
device = ('cuda' if torch.cuda.is_available() else 'cpu')
device

'cuda'

In [8]:
base_dir = '/content/My Dataset'
train_path = '/content/My Dataset/train'
test_path = '/content/My Dataset/test'

In [10]:
classes = os.listdir(train_path)

In [11]:
classes

['Pothole', 'Plain']

In [12]:
num_classes = len(classes)
num_classes

2

In [27]:
data_transform = {
    'train' : transforms.Compose([
         transforms.RandomResizedCrop(224),
         transforms.RandomHorizontalFlip(),
         transforms.ToTensor(),
         transforms.Normalize(mean=[0.5, 0.5, 0.5],
         std=[0.5, 0.5, 0.5]),
         ]),
    
     'test'  : transforms.Compose([
               transforms.Resize((224,224)),
               transforms.ToTensor(),
               transforms.Normalize(mean=[0.5, 0.5, 0.5],
               std=[0.5, 0.5, 0.5]),
         ])
     }

In [28]:
image_datasets = {x:datasets.ImageFolder(os.path.join(base_dir,x), transform=data_transform[x]) for x in ['train','test']}

In [29]:
data_loaders = {x:DataLoader(image_datasets[x], batch_size=16, shuffle=True) for x in ['train','test']}

In [30]:
dataset_sizes = {x:len(image_datasets[x]) for x in ['train','test']}

In [31]:
dataset_sizes['test']

16

In [32]:
for images, labels in data_loaders['train']:
  print(images.shape)
  break

torch.Size([16, 3, 224, 224])


In [33]:
for images, labels in data_loaders['test']:
  print(images.shape)
  break

torch.Size([16, 3, 224, 224])


In [34]:
model = models.vgg19_bn(pretrained=True)
model

Downloading: "https://download.pytorch.org/models/vgg19_bn-c79401a0.pth" to /root/.cache/torch/hub/checkpoints/vgg19_bn-c79401a0.pth


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace=True)
    (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU(inplace=True)
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (14): Conv2d(128, 256

In [35]:
for param in model.parameters():
    param.requires_grad = False

In [36]:
model.classifier = nn.Sequential(nn.Linear(25088,num_classes),
                                 nn.LogSoftmax(dim=1)) 

In [37]:
model

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace=True)
    (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU(inplace=True)
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (14): Conv2d(128, 256

In [38]:
def train_model(model, criterion, optimizer, epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(1, epochs+1):
        print('Epoch {}/{}'.format(epoch, epochs))
        print('-' * 10)

       
        for phase in ['train', 'test']:
            if phase == 'train':
                model.train()  
            else:
                model.eval()   

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in data_loaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()

                
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            if phase == 'test' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best test Acc: {:4f}'.format(best_acc))

    model.load_state_dict(best_model_wts)
    return model

In [39]:
criterion = nn.NLLLoss()

optimizer = optim.Adam(model.classifier.parameters(), lr = 0.001)

model.to(device); 

In [40]:
model_ft = train_model(model, criterion, optimizer, epochs=10)

Epoch 1/10
----------


/usr/local/lib/python3.6/dist-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


train Loss: 0.2245 Acc: 0.9087
test Loss: 0.2226 Acc: 0.8750

Epoch 2/10
----------
train Loss: 0.1709 Acc: 0.9405
test Loss: 0.4459 Acc: 0.7500

Epoch 3/10
----------
train Loss: 0.1511 Acc: 0.9461
test Loss: 0.4517 Acc: 0.8125

Epoch 4/10
----------
train Loss: 0.1488 Acc: 0.9488
test Loss: 0.5496 Acc: 0.8125

Epoch 5/10
----------
train Loss: 0.1485 Acc: 0.9488
test Loss: 0.3765 Acc: 0.8125

Epoch 6/10
----------
train Loss: 0.1202 Acc: 0.9571
test Loss: 0.3425 Acc: 0.8125

Epoch 7/10
----------
train Loss: 0.1587 Acc: 0.9488
test Loss: 0.2165 Acc: 0.8125

Epoch 8/10
----------
train Loss: 0.1290 Acc: 0.9585
test Loss: 0.4673 Acc: 0.7500

Epoch 9/10
----------
train Loss: 0.1187 Acc: 0.9640
test Loss: 0.3988 Acc: 0.8125

Epoch 10/10
----------
train Loss: 0.1216 Acc: 0.9585
test Loss: 0.3038 Acc: 0.8750

Training complete in 3m 26s
Best test Acc: 0.875000
